In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pathlib
import sys

import pandas as pd

from cev.widgets import Embedding, compare

### Ozette vs Regular

In [3]:
data_dir = pathlib.Path.cwd() / ".." / "data" / "mair-2022-ismb"
name = "TISSUE_138_samples_FM96_OM138_035_CD45_live_fcs_110595"

a = Embedding.from_ozette(df=pd.read_parquet(data_dir / f"{name}_umap.parquet"))
b = Embedding.from_ozette(
    df=pd.read_parquet(data_dir / f"{name}_umap_annotated.parquet")
)

In [4]:
w = compare(a, b)
w

## IFN-γ-independent immune markers of Mycobacterium tuberculosis exposure


In [5]:
import pathlib

import pandas as pd


def load_ics_example(subset: pd.DataFrame, data_dir="~/data/ICS_external_analysis"):
    data_dir = pathlib.Path(data_dir)
    for index, row in subset.iterrows():
        df = pd.read_parquet(data_dir / f"{row['oz_id']}_embedding_output.parquet")
        yield Embedding.from_ozette(df, robust_only=True)


def select_subset(
    metadata_file="../data/ics/ics_experiment_metadata_table.parquet",
):
    df = pd.read_parquet(metadata_file)
    # df = df[df.arm_or_cohort_name == "LTBI"]
    options = [
        # 'TB Lysate',
        "Peptide Pool 1",
        # 'DMSO',
        # 'SEB',
        # 'Peptide Pool 2'
    ]
    df = df[df.Treatment.isin(options)]
    n = 1
    return (
        df.groupby(["Treatment", "arm_or_cohort_name"])
        .sample(n=n, random_state=123)
        .reset_index(drop=True)
    )


subset = select_subset()
subset

,name,visit_name,biosample_id,visit_id,expsample_id,arm_or_cohort_id,Treatment,arm_or_cohort_name,subject_id,oz_id
0,68bd69b2-4ef6-4628-b87b-06e243a01f94,V0,1375,329,1483,323,Peptide Pool 1,LTBI,812,OZEXPSMPL_2022
1,d5c7e837-8014-429c-827e-ef05a19c6efb,V0,1284,329,1381,321,Peptide Pool 1,RSTR,721,OZEXPSMPL_2054


In [6]:
samplea, sampleb = load_ics_example(subset)

In [7]:
print(
    f"left: {subset.arm_or_cohort_name.loc[0]}, right: {subset.arm_or_cohort_name.loc[1]}"
)
w = compare(samplea, sampleb)
w

left: LTBI, right: RSTR
